# Import libraries

In [ ]:
!pip install bitsandbytes>=0.39.0 accelerate>=0.20.0
!pip install transformers

In [ ]:
import inspect
import os
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from PIL import Image
from transformers import Blip2Processor, Blip2ForConditionalGeneration

# Instantiate model

In [ ]:
# Instantiate BLIP-2 model. via
# https://huggingface.co/Salesforce/blip2-flan-t5-xxl
processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xxl")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xxl", device_map="auto", load_in_8bit=True)

# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create directory to store inferences

In [ ]:
os.makedirs('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions', exist_ok=True)

# Import datasets

## DISARM

In [ ]:
DISARM_test_all_captioned_Qwen_VL_Chat = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/DISARM_test_all_captioned_Qwen-VL-Chat.csv')
DISARM_test_all_captioned_InternLM_XComposer = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/DISARM_test_all_captioned_InternLM-XComposer.csv')
DISARM_test_all_captioned_llava = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/DISARM_test_all_captioned_llava-v1.5-13b.csv')
DISARM_test_all_captioned_BLIP_2 = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/DISARM_test_all_captioned_BLIP-2.csv')

# Set prompts and define a function to call the model

In [ ]:
def prompt_with_caption_truefalse(entity, caption):
    return inspect.cleandoc(f"""
    Description of this meme: \"\"\"
    {caption}
    \"\"\"
    Does this meme harmfully target {entity}?
    true: a social entity is subjected to some form of ill-treatment such as mental abuse, psycho-physiological injury, proprietary damage, emotional disturbance, or public image damage, based on their background (bias, social background, educational background, etc.) by a meme author.
    false: any benign mention (or depiction) of a social entity via humour, limerick, harmless pun or any content that does not cause distress.
    Constraint: Without using any other words, answer either true or false.""")

In [ ]:
# Use BLIP-2 for the inference. via
# https://huggingface.co/Salesforce/blip2-flan-t5-xxl
def get_prediction(image, prompt):
    raw_image = Image.open(image).convert('RGB')
    inputs = processor(raw_image, prompt, return_tensors="pt").to("cuda")
    out = model.generate(**inputs, max_new_tokens=30)
    return processor.decode(out[0], skip_special_tokens=True)

# Define function to clean responses

In [ ]:
def remap(x):
    x = x.lower()
    if x in {'true', 'true.', 'answer: true', 'answer: true.'}:
        return 'harmful'
    elif x in {'false', 'false.', 'answer: false', 'answer: false.'}:
        return 'not harmful'
    else:
        return None

# Call the `get_prediction` function and save inferences

GROUNDING [ABSENT] & PROMPT [CAPTION_TRUEFALSE] & CAPTION [Qwen_VL_Chat]

In [ ]:
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat = DISARM_test_all_captioned_Qwen_VL_Chat.copy(deep=True)
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat_images = DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['image'].values
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat_entities = DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['target'].values
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat_captions = DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['caption'].values
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = [get_prediction(image, prompt_with_caption_truefalse(entity, caption)) for image, entity, caption in zip(DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat_images, DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat_entities, DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat_captions)]
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/DISARM_test_all_grounding[ABSENT]_caption[Qwen-VL-Chat]_prompt[CAPTION_TRUEFALSE]_prediction[BLIP-2].csv', index=False)
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].apply(lambda x: remap(x))
print(DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].value_counts())
print(DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].isna().sum())
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].apply(lambda x: x if x is not None else np.random.choice(['harmful', 'not harmful']))
print(f1_score(DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].values, labels=['harmful', 'not harmful'], average='macro'))
print(classification_report(DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].values, labels=['harmful', 'not harmful']))

harmful        337
not harmful    275
Name: prediction, dtype: int64
0
0.5650413154429375
              precision    recall  f1-score   support

     harmful       0.58      0.61      0.59       316
 not harmful       0.56      0.52      0.54       296

    accuracy                           0.57       612
   macro avg       0.57      0.57      0.57       612
weighted avg       0.57      0.57      0.57       612



GROUNDING [ABSENT] & PROMPT [CAPTION_TRUEFALSE] & CAPTION [InternLM_XComposer]

In [ ]:
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer = DISARM_test_all_captioned_InternLM_XComposer.copy(deep=True)
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer_images = DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['image'].values
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer_entities = DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['target'].values
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer_captions = DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['caption'].values
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = [get_prediction(image, prompt_with_caption_truefalse(entity, caption)) for image, entity, caption in zip(DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer_images, DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer_entities, DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer_captions)]
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/DISARM_test_all_grounding[ABSENT]_caption[InternLM-XComposer]_prompt[CAPTION_TRUEFALSE]_prediction[BLIP-2].csv', index=False)
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].apply(lambda x: remap(x))
print(DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].value_counts())
print(DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].isna().sum())
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].apply(lambda x: x if x is not None else np.random.choice(['harmful', 'not harmful']))
print(f1_score(DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].values, labels=['harmful', 'not harmful'], average='macro'))
print(classification_report(DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].values, labels=['harmful', 'not harmful']))

Token indices sequence length is longer than the specified maximum sequence length for this model (676 > 512). Running this sequence through the model will result in indexing errors


not harmful    346
harmful        266
Name: prediction, dtype: int64
0
0.5381057905385875
              precision    recall  f1-score   support

     harmful       0.56      0.47      0.52       316
 not harmful       0.52      0.61      0.56       296

    accuracy                           0.54       612
   macro avg       0.54      0.54      0.54       612
weighted avg       0.54      0.54      0.54       612



GROUNDING [ABSENT] & PROMPT [CAPTION_TRUEFALSE] & CAPTION [llava]

In [ ]:
DISARM_test_all_grounding_absent_prompt_with_caption_llava = DISARM_test_all_captioned_llava.copy(deep=True)
DISARM_test_all_grounding_absent_prompt_with_caption_llava_images = DISARM_test_all_grounding_absent_prompt_with_caption_llava['image'].values
DISARM_test_all_grounding_absent_prompt_with_caption_llava_entities = DISARM_test_all_grounding_absent_prompt_with_caption_llava['target'].values
DISARM_test_all_grounding_absent_prompt_with_caption_llava_captions = DISARM_test_all_grounding_absent_prompt_with_caption_llava['caption'].values
DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'] = [get_prediction(image, prompt_with_caption_truefalse(entity, caption)) for image, entity, caption in zip(DISARM_test_all_grounding_absent_prompt_with_caption_llava_images, DISARM_test_all_grounding_absent_prompt_with_caption_llava_entities, DISARM_test_all_grounding_absent_prompt_with_caption_llava_captions)]
DISARM_test_all_grounding_absent_prompt_with_caption_llava.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/DISARM_test_all_grounding[ABSENT]_caption[llava-v1.5-13b]_prompt[CAPTION_TRUEFALSE]_prediction[BLIP-2].csv', index=False)
DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'].apply(lambda x: remap(x))
print(DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'].value_counts())
print(DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'].isna().sum())
DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'].apply(lambda x: x if x is not None else np.random.choice(['harmful', 'not harmful']))
print(f1_score(DISARM_test_all_grounding_absent_prompt_with_caption_llava['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'].values, labels=['harmful', 'not harmful'], average='macro'))
print(classification_report(DISARM_test_all_grounding_absent_prompt_with_caption_llava['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'].values, labels=['harmful', 'not harmful']))

not harmful    483
harmful        129
Name: prediction, dtype: int64
0
0.4968484516305838
              precision    recall  f1-score   support

     harmful       0.62      0.25      0.36       316
 not harmful       0.51      0.83      0.63       296

    accuracy                           0.53       612
   macro avg       0.57      0.54      0.50       612
weighted avg       0.57      0.53      0.49       612



GROUNDING [ABSENT] & PROMPT [CAPTION_TRUEFALSE] & CAPTION [BLIP-2]

In [ ]:
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2 = DISARM_test_all_captioned_BLIP_2.copy(deep=True)
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2_images = DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['image'].values
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2_entities = DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['target'].values
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2_captions = DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['caption'].values
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = [get_prediction(image, prompt_with_caption_truefalse(entity, caption)) for image, entity, caption in zip(DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2_images, DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2_entities, DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2_captions)]
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/DISARM_test_all_grounding[ABSENT]_caption[BLIP_2]_prompt[CAPTION_TRUEFALSE]_prediction[BLIP-2].csv', index=False)
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'].apply(lambda x: remap(x))
print(DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'].value_counts())
print(DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'].isna().sum())
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'].apply(lambda x: x if x is not None else np.random.choice(['harmful', 'not harmful']))
print(f1_score(DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'].values, labels=['harmful', 'not harmful'], average='macro'))
print(classification_report(DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'].values, labels=['harmful', 'not harmful']))

not harmful    444
harmful        168
Name: prediction, dtype: int64
0
0.5147196783560419
              precision    recall  f1-score   support

     harmful       0.60      0.32      0.41       316
 not harmful       0.51      0.77      0.62       296

    accuracy                           0.54       612
   macro avg       0.55      0.54      0.51       612
weighted avg       0.56      0.54      0.51       612

